In [1]:
import rdflib
import pandas as pd
import numpy as np
import csv
import sklearn.metrics


In [2]:
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [3]:
WDT["P58"]

rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P58')

In [5]:
graph = rdflib.Graph().parse('../../data/14_graph.nt', format='turtle')

In [10]:
entity_emb = np.load('../../data/entity_embeds.npy')
relation_emb = np.load('../../data/relation_embeds.npy')

with open('../../data/entity_ids.del', 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open('../../data/relation_ids.del', 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

In [11]:
ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

In [12]:
# let's see what our graph thinks the occupation of Jean Van Hamme is
professions = set(graph.query('''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>
    
    SELECT ?obj ?lbl WHERE {
        ?ent rdfs:label "Jean Van Hamme"@en .
        ?ent wdt:P106 ?obj .
        ?obj rdfs:label ?lbl .
    }
    '''))
{ent[len(WD):]: str(lbl) for ent, lbl in professions}

{'Q329737': 'butcher'}

In [13]:
# "Jean Van Hamme" entity
head = entity_emb[ent2id[WD.Q428160]]
# "occupation" relation
pred = relation_emb[rel2id[WDT.P106]]
# add vectors according to TransE scoring function.
lhs = head + pred
# compute distance to *any* entity
dist = sklearn.metrics.pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
# find most plausible entities
most_likely = dist.argsort()
# compute ranks of entities
ranks = dist.argsort().argsort()

In [14]:
# what would be more plausible occupations?
pd.DataFrame([
    (id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], rank+1)
    for rank, idx in enumerate(most_likely[:10])],
    columns=('Entity', 'Label', 'Score', 'Rank'))

,Entity,Label,Score,Rank
0,Q36180,writer,3223.151367,1
1,Q33999,actor,3382.129883,2
2,Q6625963,novelist,3397.582520,3
3,Q1930187,journalist,3492.735352,4
4,Q4610556,model,3549.961182,5
5,Q639669,musician,3616.397949,6
6,Q67311526,Obalky knih.cz,3628.230957,7
7,Q150,French,3667.630615,8
8,Q1028181,painter,3694.667480,9
9,Q245068,comedian,3744.303955,10


# Recovering

The Masked Gang: Cyprus (Q7750525) does not containt screenwriter (P58):

In [15]:
set(graph.objects(WD.Q7750525, WDT.P58))

set()

In [16]:
from sklearn.metrics import pairwise_distances

# Let's see if we can recover this from embeddings...

# set the head entity to bridge
head = entity_emb[ent2id[WD.Q5058838]]
# let's try with wdt:P279 (subClassOf) first
pred = relation_emb[rel2id[WDT.P58]]
# combine according to the TransE scoring function
lhs = head + pred
# compute distance to *any* entity
dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
# find most plausible tails
most_likely = dist.argsort()
# show most likely entities
pd.DataFrame([
    (id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], rank+1)
    for rank, idx in enumerate(most_likely[:10])],
    columns=('Entity', 'Label', 'Score', 'Rank'))



,Entity,Label,Score,Rank
0,Q5058838,Cengiz Küçükayvaz,4532.009766,1
1,Q6812382,Melih Ekener,4848.929199,2
2,Q12812724,Zübeyr Şaşmaz,4878.968750,3
3,Q6085709,Fırat Yükselir,4880.088379,4
4,Q6080728,Ugur Icbak,4890.920410,5
5,Q25476953,Sacide Taşaner,4901.976562,6
6,Q3328043,Murat Aslan,4906.344238,7
7,Q1956183,Muzaffer Özdemir,4913.630371,8
8,Q6089106,Gülsüm Kamu,4920.818359,9
9,Q382114,Mehmet Ali Erbil,4922.416504,10


# Entity Similarity

In [40]:
# which entities are similar to "Harry Potter and the Goblet of Fire"
ent = ent2id[WD.Q102225]
# we compare the embedding of the query entity to all other entity embeddings
print(ent)
dist = sklearn.metrics.pairwise_distances(entity_emb[ent].reshape(1, -1), entity_emb).reshape(-1)
# order by plausibility
most_likely = dist.argsort()[1]
entity = id2ent[most_likely]
print(entity)
# pd.DataFrame([
#     (
#         id2ent[idx][len(WD):], # qid
#         ent2lbl[id2ent[idx]],  # label
#         dist[idx],             # score
#         rank+1,                # rank
#     )
#     for rank, idx in enumerate(most_likely[:15])],
#     columns=('Entity', 'Label', 'Score', 'Rank'))

15725
http://www.wikidata.org/entity/Q102235


In [80]:
from collections import Counter

def recommendations_embeddings(ents: list) -> list:
        # which entities are similar to "Harry Potter and the Goblet of Fire"
        ents = [ent2id[WD[ent]] for ent in ents]
        
        # we compare the embedding of the query entity to all other entity embeddings
        recommendations = []
        for ent in ents:
            dist = sklearn.metrics.pairwise_distances(
                entity_emb[ent].reshape(1, -1), entity_emb
            ).reshape(-1)
            # order by plausibility
            most_likely = dist.argsort()[1:6]
            recommendations += ([str(id2ent[movie]) for movie in most_likely])
        counts = Counter(recommendations).most_common(2)
        first_rec = counts[0][0]
        second_rec = counts[1][0]
        return first_rec, second_rec

In [81]:
print(recommendations_embeddings(["Q102225", "Q102235"]))


15725
3604
('http://www.wikidata.org/entity/Q161678', 'http://www.wikidata.org/entity/Q161687')


In [82]:
print("x" + "y")

xy
